# 1. Start Elasticsearch Server

In [ ]:
#Execute elasticsearch.bat file under elasticsearch-8.13.4\bin

In [4]:
#Installing and Downloading Elasticsearch
#Download: https://www.elastic.co/downloads/elasticsearch

#Open cmd:
#C:\Users\Abhishek_Jaiswal>cd C:\Users\Abhishek_Jaiswal\Downloads\elasticsearch-8.13.4\bin
#C:\Users\Abhishek_Jaiswal\Downloads\elasticsearch-8.13.4\bin>elasticsearch
#or 
#run elasticsearch.bat file under bin.

#https://localhost:9200/
#username: elastic
#password: vfkWRFChlFKFBDPz*0y8

#Learning:
#Link:https://www.youtube.com/watch?v=K-BBol9hQ54

# 2. Reading Data and Embedding Creation

## Prepare the data

In [5]:
import pandas as pd
df = pd.read_csv("myntra_products_catalog.csv").loc[:499]
df.head(2)

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige


In [6]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           468
                                                                                   True             32
Name: count, dtype: int64

In [7]:
df.fillna("None", inplace=True)

## Convert the relevant field to Vector using BERT model

In [17]:
# Embeddings
#https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [9]:
df["DescriptionVector"] = df["Description"].apply(lambda x: model.encode(x))

In [14]:
print("Shape: ",df.shape)
df.head(2)

Shape:  (500, 9)


,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,DescriptionVector
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,"[0.027645793, -0.0026342259, -0.0035884194, 0...."
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,"[-0.02466072, -0.02875537, -0.020332465, 0.034..."


# 3. Enabling VD

In [15]:
from elasticsearch import Elasticsearch
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","vfkWRFChlFKFBDPz*0y8"),
    ca_certs="C:/Users/Abhishek_Jaiswal/Downloads/elasticsearch-8.13.4/config/certs/http_ca.crt"
)
es.ping()

True

In [16]:
df.head(2)

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,DescriptionVector
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,"[0.027645793, -0.0026342259, -0.0035884194, 0...."
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,"[-0.02466072, -0.02875537, -0.020332465, 0.034..."


## Create new index in ElasticSearch!

In [18]:
indexMapping = {
    "properties":{
        "ProductID":{
            "type":"long"
        },
        "ProductName":{
            "type":"text"
        },
        "ProductBrand":{
            "type":"text"
        },
        "Gender":{
            "type":"text"
        },
        "Price (INR)":{
            "type":"long"
        },
        "NumImages":{
            "type":"long"
        },
        "Description":{
            "type":"text"
        },
        "PrimaryColor":{
            "type":"text"
        },
        "DescriptionVector":{
            "type":"dense_vector",
            "dims": 768,
            "index":True,
            "similarity": "l2_norm" #similarity calculations
        }

    }
}

## similarity calculations type
## L2 Norm (Euclidean Distance): Best when vector magnitude matters.
## Cosine Similarity: Focuses on the angle between vectors, useful for text or embeddings.
## Dot Product: Projects one vector onto another, useful when vectors are normalized.
## L1 Norm (Manhattan Distance): Uniform treatment of dimension differences, good for sparse data.
## Jaccard Similarity: Measures the overlap between sets.
## Hamming Distance: Counts differing bits between binary strings.

In [20]:
from indexMapping import indexMapping

es.indices.create(index="all_products_2", mappings=indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_products_2'})

## Ingest the data into index

In [21]:
record_list = df.to_dict("records")
record_list[0]

{'ProductID': 10017413,
 'ProductName': 'DKNY Unisex Black & Grey Printed Medium Trolley Bag',
 'ProductBrand': 'DKNY',
 'Gender': 'Unisex',
 'Price (INR)': 11745,
 'NumImages': 7,
 'Description': 'Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer',
 'PrimaryColor': ' Black',
 'DescriptionVector': array([ 2.76457928e-02, -2.63422588e-03, -3.58841941e-03,  5.13588227e-02,
         3.09660714e-02,  1.40506746e-02,  7.27058807e-03,  3.13871130e-02,
        -6.23787493e-02, -3.82873393e-03,  3.15214060e-02,  7.55473003e-02,
         2.12639500e-03,  4.64893542e-02,  5.07448688e-02, -1.71942487e-02,
         1.22891450e-02, -1.95682421e-02, -9.6

In [22]:
for record in record_list:
    try:
        es.index(index="all_products_2", document=record, id=record["ProductID"])
    except Exception as e:
        print(e)

In [24]:
es.count(index="all_products_2")

ObjectApiResponse({'count': 500, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

# 4. Search the data

In [25]:
input_keyword = "Blue Shoes"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "DescriptionVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 500, 
}

res = es.knn_search(index="all_products_2", knn=query , source=["ProductName","Description"])
res["hits"]["hits"]

C:\Users\Abhishek_Jaiswal\AppData\Local\Temp\ipykernel_19832\3672793080.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_products_2", knn=query , source=["ProductName","Description"])


[{'_index': 'all_products_2',
  '_id': '10018013',
  '_score': 0.6142942,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}},
 {'_index': 'all_products_2',
  '_id': '10018075',
  '_score': 0.6142942,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}}]

# Semantic Caching

### Overview of Semantic Cache

**Semantic caching** is a caching strategy that involves storing the results of previously computed queries along with their associated data (often in the form of embeddings). It utilizes the semantic similarity between queries to improve retrieval efficiency. Instead of recomputing results for every query, a semantic cache allows for quick access to cached responses that are semantically similar to the current query.

#### Key Components:

1. **Embeddings**: 
   - Queries and their responses are converted into high-dimensional vectors (embeddings) using models like Sentence Transformers. This allows for the measurement of similarity between different queries based on their content.

2. **Indexing**: 
   - Cached responses are indexed in a database (e.g., Elasticsearch) along with their embeddings, making it easy to retrieve them based on similarity scores.

3. **Retrieval Mechanism**: 
   - When a new query is received, its embedding is computed, and a search is performed against the cached embeddings. The most similar cached responses are retrieved based on a similarity measure (e.g., cosine similarity).

### Advantages of Semantic Cache

1. **Improved Efficiency**:
   - Reduces the need for repeated computation by quickly returning cached responses for semantically similar queries, significantly speeding up response times.

2. **Reduced Latency**:
   - As the time taken to retrieve a cached response is generally lower than generating a fresh response, this leads to lower latency, especially in high-demand applications.

3. **Resource Optimization**:
   - By avoiding redundant computations, semantic caching helps in optimizing resource utilization, such as CPU and memory usage, which is particularly beneficial in cloud-based environments.

4. **Enhanced User Experience**:
   - Users receive responses faster, improving the overall experience, especially in applications like chatbots, customer support, and information retrieval systems.

5. **Flexibility with Fresh Data**:
   - While cached responses are often used, the system can still generate fresh responses when necessary. This allows it to balance between using cached data and accessing the latest information.

6. **Scalability**:
   - Semantic caching can be easily scaled. As more queries are processed and more responses are cached, the system can handle larger volumes of queries efficiently.

7. **Dynamic Adaptation**:
   - As user queries evolve, the semantic cache can adapt by learning from new queries and responses, continuously improving its effectiveness.

### Use Cases

- **Natural Language Processing (NLP)**: For chatbots and virtual assistants to quickly answer frequently asked questions.
- **Search Engines**: To speed up responses to common search queries by caching results.
- **Recommendation Systems**: To provide quick recommendations based on previously stored preferences or similar user behaviors.
- **Knowledge Management Systems**: For quickly retrieving information based on user inquiries in corporate environments.

In summary, semantic caching provides a sophisticated method for improving the efficiency and responsiveness of systems that require rapid access to information. Its ability to leverage the semantic relationships between queries and responses makes it a powerful tool in modern data-driven applications.

# 1. Create and Save Dataset

In [48]:
import pandas as pd

# Step 1: Create a dataset with 20 examples and save it to a CSV file
def create_and_save_dataset(file_path):
    data = {
        'query': [
            "What is the capital of France?",
            "What is the largest planet in our solar system?",
            "How many continents are there?",
            "What is the boiling point of water?",
            "Who wrote 'To Kill a Mockingbird'?",
            "What is the currency of Japan?",
            "Who painted the Mona Lisa?",
            "What is the tallest mountain in the world?",
            "How many bones are in the human body?",
            "What is the chemical symbol for gold?",
            "What year did the Titanic sink?",
            "Who is known as the father of modern physics?",
            "What is the fastest land animal?",
            "What is the square root of 64?",
            "Who discovered penicillin?",
            "What is the capital of Australia?",
            "What is the largest ocean on Earth?",
            "Who was the first president of the United States?",
            "What is the speed of light?",
            "What is the main language spoken in Brazil?"
        ],
        'response': [
            "The capital of France is Paris.",
            "The largest planet in our solar system is Jupiter.",
            "There are seven continents.",
            "The boiling point of water is 100 degrees Celsius.",
            "Harper Lee wrote 'To Kill a Mockingbird'.",
            "The currency of Japan is the Yen.",
            "Leonardo da Vinci painted the Mona Lisa.",
            "The tallest mountain in the world is Mount Everest.",
            "There are 206 bones in the human body.",
            "The chemical symbol for gold is Au.",
            "The Titanic sank in 1912.",
            "Albert Einstein is known as the father of modern physics.",
            "The fastest land animal is the cheetah.",
            "The square root of 64 is 8.",
            "Alexander Fleming discovered penicillin.",
            "The capital of Australia is Canberra.",
            "The largest ocean on Earth is the Pacific Ocean.",
            "George Washington was the first president of the United States.",
            "The speed of light is approximately 299,792 kilometers per second.",
            "The main language spoken in Brazil is Portuguese."
        ]
    }
    
    df = pd.DataFrame(data)
    df.to_csv(file_path, index=False)
    print(f"Dataset saved to {file_path}")

# Example usage
dataset_path = 'query_response_dataset.csv'
create_and_save_dataset(dataset_path)

Dataset saved to query_response_dataset.csv


# 2. Connect to Elasticsearch

In [49]:
from elasticsearch import Elasticsearch

# Step 2: Connect to Elasticsearch
def connect_to_elasticsearch():
    es = Elasticsearch(
        "https://localhost:9200",
        basic_auth=("elastic", "vfkWRFChlFKFBDPz*0y8"),
        ca_certs="C:/Users/Abhishek_Jaiswal/Downloads/elasticsearch-8.13.4/config/certs/http_ca.crt"
    )
    if es.ping():
        print("Connected to Elasticsearch.")
    else:
        raise ConnectionError("Could not connect to Elasticsearch.")
    return es

# Example usage
es = connect_to_elasticsearch()

Connected to Elasticsearch.


# 3. Create an Index in Elasticsearch

In [51]:
# Step 3: Create an index in Elasticsearch
def create_index(es, index_name):
    es.indices.create(index=index_name, ignore=400)  # Ignore if index already exists
    print(f"Index '{index_name}' created or already exists.")

# Example usage
index_name = 'semantic_cache_2'
create_index(es, index_name)

C:\Users\Abhishek_Jaiswal\AppData\Local\Temp\ipykernel_19832\3250227610.py:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, ignore=400)  # Ignore if index already exists


Index 'semantic_cache_2' created or already exists.


# 4. Load the Sentence Transformer Model

In [52]:
from sentence_transformers import SentenceTransformer

# Step 4: Load the Sentence Transformer model
def load_model(model_name='all-MiniLM-L6-v2'):
    return SentenceTransformer(model_name)

# Example usage
model = load_model()

# 5. Store Query-Response Pairs in Elasticsearch

In [53]:
# Step 5: Store query-response pairs in Elasticsearch
def store_query_response(es, model, index_name, query, response):
    query_embedding = model.encode(query).tolist()  # Convert to list for JSON serialization
    doc = {
        'query': query,
        'response': response,
        'embedding': query_embedding
    }
    es.index(index=index_name, document=doc)
    print(f"Stored query-response pair: '{query}' -> '{response}'")

# Example usage for storing initial dataset (this can be done in the main execution)
import pandas as pd

# Read the dataset and store it in Elasticsearch
df = pd.read_csv(dataset_path)
for _, row in df.iterrows():
    store_query_response(es, model, index_name, row['query'], row['response'])

Stored query-response pair: 'What is the capital of France?' -> 'The capital of France is Paris.'
Stored query-response pair: 'What is the largest planet in our solar system?' -> 'The largest planet in our solar system is Jupiter.'
Stored query-response pair: 'How many continents are there?' -> 'There are seven continents.'
Stored query-response pair: 'What is the boiling point of water?' -> 'The boiling point of water is 100 degrees Celsius.'
Stored query-response pair: 'Who wrote 'To Kill a Mockingbird'?' -> 'Harper Lee wrote 'To Kill a Mockingbird'.'
Stored query-response pair: 'What is the currency of Japan?' -> 'The currency of Japan is the Yen.'
Stored query-response pair: 'Who painted the Mona Lisa?' -> 'Leonardo da Vinci painted the Mona Lisa.'
Stored query-response pair: 'What is the tallest mountain in the world?' -> 'The tallest mountain in the world is Mount Everest.'
Stored query-response pair: 'How many bones are in the human body?' -> 'There are 206 bones in the human bo

# 6. Retrieve Response from Elasticsearch

In [54]:
# Step 6: Retrieve response from Elasticsearch
def retrieve_response(es, model, index_name, query):
    query_embedding = model.encode(query).tolist()
    search_query = {
        "script_score": {
            "query": {
                "match_all": {}
            },
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                "params": {
                    "query_vector": query_embedding
                }
            }
        }
    }
    response = es.search(index=index_name, query=search_query, size=1)  # Limit to the top match
    if response['hits']['hits']:
        return response['hits']['hits'][0]['_source']['response']
    else:
        return None

# Example usage
response = retrieve_response(es, model, index_name, "What is the fastest land animal?")
print("Retrieved Response:", response)

Retrieved Response: The fastest land animal is the cheetah.


# 7. Process the Query

In [55]:
# Step 7: Process the query
def process_query(es, model, index_name, query):
    cached_response = retrieve_response(es, model, index_name, query)
    if cached_response:
        return cached_response
    else:
        fresh_response = f"Fresh response generated for: {query}"  # Placeholder logic for generating a fresh response
        store_query_response(es, model, index_name, query, fresh_response)
        return fresh_response

# Example usage
result = process_query(es, model, index_name, "What is the capital city of France?")
print("Processed Query Response:", result)

Processed Query Response: The capital of France is Paris.
